In [29]:
import pdfplumber
import os
from datetime import datetime
from pymongo import MongoClient


# Starter code for extracting pdf data with pdf plumber
document_maps = []
for dir in os.walk('/Users/jaredperez/Desktop/Langchain-AI-lecture/AI-Agent-Lecture/data'):
    files = dir[2]
    for file in files:
        if file != '.DS_Store':
            pdf_map = {}
            with pdfplumber.open(f'/Users/jaredperez/Desktop/Langchain-AI-lecture/AI-Agent-Lecture/data/{file}') as pdf:
                first_page = pdf.pages[0]
                document_arr = [pdf.pages[i].extract_text() for i in range(len(pdf.pages))]
                print(first_page.extract_text())
                pdf_map['Title'] = file
                pdf_map['text'] = document_arr
                pdf_map['Num Pages'] = len(document_arr)
                pdf_map['source'] = 'internet'
                pdf_map['Date of Upload'] = datetime.now()


                ### Metadata to be solved/fixed later
                document_maps.append(pdf_map)
                

Redox Biology 35 (2020) 101499
ContentslistsavailableatScienceDirect
Redox Biology
journal homepage: www.elsevier.com/locate/redox
Redox basis of exercise physiology
T
N.V. Margaritelisa,b,∗∗ , V. Paschalisc, A.A. Theodoroud, A. Kyparosa, M.G. Nikolaidisa,∗
aDepartmentofPhysicalEducationandSportScienceatSerres,AristotleUniversityofThessaloniki,Thessaloniki,Greece
bDialysisUnit,424GeneralMilitaryHospitalofThessaloniki,Thessaloniki,Greece
cSchoolofPhysicalEducationandSportScience,NationalandKapodistrianUniversityofAthens,Athens,Greece
dDepartmentofLifeSciences,SchoolofSciences,EuropeanUniversityCyprus,Nicosia,Cyprus
ARTICLE INFO ABSTRACT
Keywords: Redoxreactionscontrolfundamentalprocessesofhumanbiology.Therefore,itissafetoassumethatthere-
Adaptations sponsesandadaptationstoexerciseare,atleastinpart,mediatedbyredoxreactions.Inthisreview,weare
Antioxidants tryingtoshowthatredoxreactionsarethebasisofexercisephysiologybyoutliningtheredoxsignalingpath-
Exercise ways that regulate four charact

## Instert All documents.

In [45]:
## Do some mapping to insert all the documents
document_maps
page_map = {}
var = document_maps[0]['text']
for i in range(len(document_maps[0]['text'])):
    page_map[i] = var[i]
page_list = [{"text": text} for _, text in page_map.items()]



client = MongoClient('mongodb://localhost:27017/')
db = client['redox']
collection = db['data4']

# Insert the list into MongoDB
collection.insert_many(page_list)

InsertManyResult([ObjectId('66b905f31fa45f6e7a8401f7'), ObjectId('66b905f31fa45f6e7a8401f8'), ObjectId('66b905f31fa45f6e7a8401f9'), ObjectId('66b905f31fa45f6e7a8401fa'), ObjectId('66b905f31fa45f6e7a8401fb'), ObjectId('66b905f31fa45f6e7a8401fc'), ObjectId('66b905f31fa45f6e7a8401fd'), ObjectId('66b905f31fa45f6e7a8401fe'), ObjectId('66b905f31fa45f6e7a8401ff'), ObjectId('66b905f31fa45f6e7a840200'), ObjectId('66b905f31fa45f6e7a840201'), ObjectId('66b905f31fa45f6e7a840202'), ObjectId('66b905f31fa45f6e7a840203'), ObjectId('66b905f31fa45f6e7a840204'), ObjectId('66b905f31fa45f6e7a840205'), ObjectId('66b905f31fa45f6e7a840206'), ObjectId('66b905f31fa45f6e7a840207'), ObjectId('66b905f31fa45f6e7a840208'), ObjectId('66b905f31fa45f6e7a840209'), ObjectId('66b905f31fa45f6e7a84020a'), ObjectId('66b905f31fa45f6e7a84020b'), ObjectId('66b905f31fa45f6e7a84020c')], acknowledged=True)

## Extract

In [46]:
from llama_index.readers.mongodb import SimpleMongoReader
from langchain.chains import LLMChain


client = MongoClient('mongodb://localhost:27017/')
host =  'localhost'
port = 27017
db = 'redox'
collection = 'data4'

query_dict = {}
field_names = ['text']

reader = SimpleMongoReader(uri = 'mongodb://localhost:27017/')
documents = reader.load_data(db, collection, field_names, query_dict = query_dict
                             )
documents

[Document(id_='bbb9e91f-fe9c-4f24-8899-6d3114274b20', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Redox Biology 35 (2020) 101499\nContentslistsavailableatScienceDirect\nRedox Biology\njournal homepage: www.elsevier.com/locate/redox\nRedox basis of exercise physiology\nT\nN.V. Margaritelisa,b,∗∗ , V. Paschalisc, A.A. Theodoroud, A. Kyparosa, M.G. Nikolaidisa,∗\naDepartmentofPhysicalEducationandSportScienceatSerres,AristotleUniversityofThessaloniki,Thessaloniki,Greece\nbDialysisUnit,424GeneralMilitaryHospitalofThessaloniki,Thessaloniki,Greece\ncSchoolofPhysicalEducationandSportScience,NationalandKapodistrianUniversityofAthens,Athens,Greece\ndDepartmentofLifeSciences,SchoolofSciences,EuropeanUniversityCyprus,Nicosia,Cyprus\nARTICLE INFO ABSTRACT\nKeywords: Redoxreactionscontrolfundamentalprocessesofhumanbiology.Therefore,itissafetoassumethatthere-\nAdaptations sponsesandadaptationstoexerciseare,atleastinpart,mediated

In [47]:
from langchain_community.document_loaders.mongodb import MongodbLoader

# Define your client, db, and collection
client = MongoClient('mongodb://localhost:27017/')
db = client['redox']
collection = db['data']

# MongoDBLoader to load documents from MongoDB
loader = MongodbLoader(
    connection_string="mongodb://localhost:27017/",
    db_name="redox",
    collection_name="data4",
)

In [49]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch
import nest_asyncio
nest_asyncio.apply()
from langchain_openai import OpenAIEmbeddings


# Attenmpt to create index, this is where error may appear
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

ImportError: cannot import name 'abatch_iterate' from 'langchain_core.utils.aiter' (/Users/jaredperez/opt/miniconda3/envs/flask_blog/lib/python3.10/site-packages/langchain_core/utils/aiter.py)